![Add a relevant banner image here](path_to_image)

# Project Title

## Overview

Short project description. Your bottom line up front (BLUF) insights.

## Business Understanding

Text here

## Data Understanding

Text here

In [3]:
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt
import kagglehub
from transformers import (
    pipeline,
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    EvalPrediction,
    EarlyStoppingCallback
)
from evaluate import load
from rouge_score import rouge_scorer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support 

In [ ]:
path = kagglehub.dataset_download("nileshmalode1/samsum-dataset-text-summarization")

print("Filepath to dataset: ", path)

In [4]:
df_samsum_train = pd.read_csv("Data/samsum-train.csv")
df_samsum_test = pd.read_csv("Data/samsum-test.csv")
df_samsum_val = pd.read_csv("Data/samsum-validation.csv")

df_samsum_train.head()

,id,dialogue,summary
0,13818513,Amanda: I baked cookies. Do you want some?\r\...,Amanda baked cookies and will bring Jerry some...
1,13728867,Olivia: Who are you voting for in this electio...,Olivia and Olivier are voting for liberals in ...
2,13681000,"Tim: Hi, what's up?\r\nKim: Bad mood tbh, I wa...",Kim may try the pomodoro technique recommended...
3,13730747,"Edward: Rachel, I think I'm in ove with Bella....",Edward thinks he is in love with Bella. Rachel...
4,13728094,Sam: hey overheard rick say something\r\nSam:...,"Sam is confused, because he overheard Rick com..."


In [5]:
# converting all data into strings
df_samsum_train['id'] = df_samsum_train['id'].astype('string')
df_samsum_train['dialogue'] = df_samsum_train['dialogue'].astype('string')
df_samsum_train['summary'] = df_samsum_train['summary'].astype('string')
df_samsum_test['id'] = df_samsum_test['id'].astype('string')
df_samsum_test['dialogue'] = df_samsum_test['dialogue'].astype('string')
df_samsum_test['summary'] = df_samsum_test['summary'].astype('string')
df_samsum_val['id'] = df_samsum_val['id'].astype('string')
df_samsum_val['dialogue'] = df_samsum_val['dialogue'].astype('string')
df_samsum_val['summary'] = df_samsum_val['summary'].astype('string')

# adding a column to show length of strings in dialogue
df_samsum_train['diag_len'] = df_samsum_train['dialogue'].str.len()
df_samsum_test['diag_len'] = df_samsum_test['dialogue'].str.len()
df_samsum_val['diag_len'] = df_samsum_val['dialogue'].str.len()

df_samsum_train.describe(), df_samsum_test.describe(), df_samsum_val.describe()

(         diag_len
 count     14731.0
 mean   511.218315
 std    402.613071
 min          31.0
 25%         216.5
 50%         401.0
 75%         694.0
 max        5492.0,
          diag_len
 count       819.0
 mean   521.598291
 std    409.387325
 min          49.0
 25%         226.0
 50%         403.0
 75%         687.5
 max        2793.0,
          diag_len
 count       818.0
 mean   499.396088
 std    403.632584
 min          57.0
 25%         208.0
 50%         385.5
 75%         688.0
 max        2950.0)

## Data Preparation
Text here

In [6]:
# loading the Falconsai tokenizer and creating TensorFlow datasets
tokenizer = AutoTokenizer.from_pretrained("Falconsai/text_summarization")
train_dataset = tokenizer(df_samsum_train['dialogue'].astype(str).tolist(), padding=True, truncation='longest_first', return_tensors="pt")
test_dataset = tokenizer(df_samsum_test['dialogue'].astype(str).tolist(), padding=True, truncation='longest_first', return_tensors="pt")
val_dataset = tokenizer(df_samsum_val['dialogue'].astype(str).tolist(), padding=True, truncation='longest_first', return_tensors="pt")

## Analysis

Text here

In [ ]:
# loading the Falconsai model
model = AutoModelForSeq2SeqLM.from_pretrained("Falconsai/text_summarization")

In [ ]:
from evaluate import load

class MetricsComputer:
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer
        self.rouge = load('rouge')
        
    def compute_metrics(self, model_outputs, labels, loss=None):
        """
        Args:
            model_outputs: logits or generated token IDs from model
            labels: reference summary token IDs
            loss: precomputed loss value (optional)
        """
        metrics = {}
        
        # 1. Loss (if provided)
        if loss is not None:
            metrics['loss'] = loss.item()
        
        # 2. Generate predictions (if logits provided)
        if model_outputs.dim() == 3:  # logits shape: [batch, seq_len, vocab]
            predictions = torch.argmax(model_outputs, dim=-1)
        else:
            predictions = model_outputs
        
        # 3. Decode to text
        decoded_preds = self.tokenizer.batch_decode(
            predictions, 
            skip_special_tokens=True
        )
        decoded_labels = self.tokenizer.batch_decode(
            labels, 
            skip_special_tokens=True
        )
        
        # 4. Compute ROUGE scores
        rouge_results = self.rouge.compute(
            predictions=decoded_preds,
            references=decoded_labels,
            use_stemmer=True
        )
        
        metrics.update({
            'rouge1': rouge_results['rouge1'],
            'rouge2': rouge_results['rouge2'],
            'rougeL': rouge_results['rougeL']
        })
        
        # 5. Token-level accuracy (optional)
        if predictions.shape == labels.shape:
            mask = labels != -100  # ignore padding
            correct = (predictions == labels) & mask
            metrics['token_accuracy'] = (correct.sum() / mask.sum()).item()
        
        return metrics

## Evaluation

### Business Insight/Recommendation 1

### Business Insight/Recommendation 2

### Business Insight/Recommendation 3

### Tableau Dashboard link

## Conclusion and Next Steps
Text here